In [72]:
import math
from typing import List

class Value:
    def __init__(self, data: float, label: str, parents: List["Value"] = None):
        self.data = data
        self.label = label
        self.grad = 0.0
        self.parents = parents if parents is not None else []
        self._backward = lambda: None

    def add(self, other: "Value") -> "Value":
        data = self.data + other.data
        out = Value(data, f"({self.label} + {other.label})", [self, other])
        
        # The derivative of z with respect to x is 1, and with respect to y is 1.
        def _backward():
            self.grad += out.grad  # dz/dx = 1
            other.grad += out.grad # dz/dy = 1
        out._backward = _backward
        return out

    def mul(self, other: "Value") -> "Value":
        data = self.data * other.data
        out = Value(data, f"({self.label} * {other.label})", [self, other])
        
        # The derivative of z with respect to x is y, and with respect to y is x.
        def _backward():
            self.grad += out.grad * other.data
            other.grad += out.grad * self.data
        out._backward = _backward
        return out

    def pow(self, other: "Value") -> "Value":
        data = self.data ** other.data
        out = Value(data, f"({self.label} ** {other.label})", [self, other])
        
        # The derivative of z with respect to x is y * (x ** (y - 1)), and with respect to y is x ** y * log(x).
        def _backward():
            self.grad += out.grad * other.data * self.data ** (other.data - 1)
            other.grad += out.grad * self.data ** other.data * math.log(self.data)
        out._backward = _backward
        return out
    
    def tanh(self) -> "Value":
        data = math.tanh(self.data)
        out = Value(data, f"tanh({self.label})", [self])
        
        # The derivative of z with respect to x is 1 - z^2.
        def _backward():
            self.grad += out.grad * (1 - out.data * out.data)
        out._backward = _backward
        return out



In [73]:
def numGrad(f, x):
    eps = 1e-6
    return (f(x+eps) - f(x-eps)) / (2 * eps)

In [74]:
def topo_sort(v: Value) -> List[Value]:
    order = []
    visited = []
    def dfs(v: Value):
        if v in visited:
            return
        visited.append(v)
        for parent in v.parents:
            dfs(parent)
        order.append(v)
    dfs(v)
    return order

def zero_grad(v: Value):
    visited = []
    def dfs(v: Value):
        if v in visited:
            return
        visited.append(v)
        for parent in v.parents:
            dfs(parent)
        v.grad = 0
    dfs(v)

def backward(v: Value):
    order = topo_sort(v)
    zero_grad(v)
    v.grad = 1.0
    for node in reversed(order):
        node._backward()

def numGrad(f, x):
    eps = 1e-6
    return (f(x+eps) - f(x-eps)) / (2 * eps)

In [75]:
x = Value(1.0, "x")
two = Value(2.0, "2")
three = Value(3.0, "3")

y = two.mul(x)
z = y.add(three)
f = z.tanh()



backward(f)

# Compare to numerical grad
got = x.grad
want = numGrad(lambda xx: math.tanh(2 * xx + 3), 1.0)

err = abs(got - want)

print(f"x.grad (backprop)={got:.6f}, (numerical)={want:.6f}, err={err:.6f}")


x.grad (backprop)=0.000363, (numerical)=0.000363, err=0.000000
